In [1]:
import pandas as pd
from sklearn.metrics import r2_score

import sys
sys.path.append("../")
from experiments.const import TARGET_COLUMNS, ALL_ZERO_TARGET_COLS, FEATURE_COLUMNS

In [2]:
import os

In [3]:
import matplotlib.pyplot as plt

In [4]:
from sklearn.metrics import r2_score

In [5]:
df_test = pd.read_parquet(r"..\input\leap-atmospheric-physics-ai-climsim\test.parquet")

In [6]:
base_dir = r"D:\kaggle\leap\output\exp303\20240715201004_exp302_1dcnn_hidden[256, 256, 256]_lr0.001_bs256_beta1_newdata_40epochs_ks5"
# base_dir = r"../output/exp042/20240629120550_exp042_1.8m_diff_i+1_i-1"

In [7]:
df1 = pd.read_parquet(fr"{base_dir}\fold0\submission.parquet")
df1_val = pd.read_parquet(fr"{base_dir}\fold0\pred_valid.parquet")

In [8]:
for fold in [1, 2, 3, 4, 5, 6, 7, 8, 9]:
    if not os.path.exists(fr"{base_dir}\fold{fold}\submission.parquet"):
        print(f"fold{fold} not exists")
        continue
    df_ = pd.read_parquet(fr"{base_dir}\fold{fold}\submission.parquet")
    df1[TARGET_COLUMNS] += df_[TARGET_COLUMNS].values
    
    df_ = pd.read_parquet(fr"{base_dir}\fold{fold}\pred_valid.parquet")
    df1_val = pd.concat([df1_val, df_], axis=0)
    
df1[TARGET_COLUMNS] /= 5

fold5 not exists
fold6 not exists
fold7 not exists
fold8 not exists
fold9 not exists


In [9]:
df_label = pd.read_parquet(r"..\input\leap-atmospheric-physics-ai-climsim\valid.parquet")

In [10]:
df_label = pd.merge(df1_val[["sample_id"]], df_label)

In [11]:
df_label[ALL_ZERO_TARGET_COLS] = 0
df1_val[ALL_ZERO_TARGET_COLS] = 0
#df2_val[ALL_ZERO_TARGET_COLS] = 0

for col in [
    "ptend_q0002_12",
    "ptend_q0002_13",
    "ptend_q0002_14",
    "ptend_q0002_15",
    "ptend_q0002_16",
    "ptend_q0002_17",
    "ptend_q0002_18",
    "ptend_q0002_19",
    "ptend_q0002_20",
    "ptend_q0002_21",
    "ptend_q0002_22",
    "ptend_q0002_23",
    "ptend_q0002_24",
    "ptend_q0002_25",
    "ptend_q0002_26",
    "ptend_q0002_27",
]:
    df_label[col] = 0
    df1_val[col] = 0
    #df2_val[col] = 0

In [12]:
score = r2_score(df_label[TARGET_COLUMNS].head(len(df1_val)), df1_val[TARGET_COLUMNS])
score

0.7955883783241725

In [13]:
import os
def create_submission(df, df_val, folder):
    
    os.makedirs(folder, exist_ok=True)
    df_val.to_parquet(f"{folder}/pred_valid.parquet")
    
    df_sub = pd.read_parquet(r"..\input\leap-atmospheric-physics-ai-climsim\sample_submission.parquet")
    df_old = pd.read_parquet("20240715_ensemble_per_target_500.parquet")
    # df_replace = pd.read_parquet("20240713014222_exp302_1dcnn_hidden[256, 256, 256]_lr0.001_bs128_beta1/submission.parquet")
    
    for i in range(28):
        target_col = f"ptend_q0002_{i}"
        feature_col = f"state_q0002_{i}"
        
        df[target_col] = df_test[feature_col] / -1200
        df[target_col] *= df_sub[target_col]

    for col in TARGET_COLUMNS:  
        score = r2_score(df_old[col], df[col])
        if score < 0.975:
            print(col, score)
            df[col] = df_old[col]        
    df.to_parquet(f"{folder}/submission.parquet")

In [14]:
create_submission(df1, df1_val, folder=os.path.basename(base_dir))

ptend_q0001_14 0.9611529203572465
ptend_q0001_15 0.9595131011878957
ptend_q0002_28 0.9244938687999528
ptend_q0003_28 0.9533294542599317
ptend_q0003_29 0.9580521598525569


In [9]:
df_ = pd.read_parquet("20240714_ensemble_per_target_lb79008.parquet")
# df_ = pd.read_parquet("20240615044912_exp031_20files_256x4_resnet_block_with_resnet3d_1d64-128-256-256_applyfreq9/submission.parquet")

In [17]:
r2_score(df_[TARGET_COLUMNS], df1[TARGET_COLUMNS])

0.9953369106948038

In [28]:
for col in TARGET_COLUMNS:
    print(col, r2_score(df_[col], df1[col]))

ptend_t_0 0.9870627377000191
ptend_t_1 0.9841084671721152
ptend_t_2 0.9982481050703101
ptend_t_3 0.999223155589274
ptend_t_4 0.999389218641732
ptend_t_5 0.9994361682965156
ptend_t_6 0.9994117915250653
ptend_t_7 0.9994149485438627
ptend_t_8 0.9994364032186466
ptend_t_9 0.9994479929923297
ptend_t_10 0.9994453978861035
ptend_t_11 0.9994355118077781
ptend_t_12 0.9994089291861011
ptend_t_13 0.9994201904995935
ptend_t_14 0.9994177740354623
ptend_t_15 0.9993416633374406
ptend_t_16 0.9982346634733711
ptend_t_17 0.9871601812261098
ptend_t_18 0.9914727106512617
ptend_t_19 0.9927435958850841
ptend_t_20 0.9936711567739546
ptend_t_21 0.9950827873785488
ptend_t_22 0.9957208116440178
ptend_t_23 0.9964975185418201
ptend_t_24 0.9970650926116287
ptend_t_25 0.997746273407676
ptend_t_26 0.9982413705905175
ptend_t_27 0.9986254886217687
ptend_t_28 0.9988413236303813
ptend_t_29 0.998972140963256
ptend_t_30 0.99902552305649
ptend_t_31 0.9990445612173356
ptend_t_32 0.9990235476982207
ptend_t_33 0.9989466244323